In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def jump_diffusion_simulation(S0, r, q, sigma, lambda_j, mu_j, sigma_j, T, N, num_sims):
    #Model: dS = (r-q) * S * dt + sigma * S * dW + S * dN(lambda_j) * dJ(mu_j,sigma_j)
    # Remember to use ito calculus  !!!!!!!
    dt = T / N
    paths = np.zeros((num_sims, N+1))
    paths[:, 0] = S0
    
    for i in range(1, N+1):
# The solution take the form: S_t = S_0 * exp( (r - q - 1/2 * sigma^2) * t + sigma * Normal(0,sqrt(t)) + Poisson(lambda_j * dt) * Normal(mu_j,sigma_j) )
        
        # Diffusion component
        dW = np.random.normal(0, np.sqrt(dt), num_sims)
        diffusion = (r - q - 0.5 * sigma**2) * dt + sigma * dW
        
        # Jump component
        dN = np.random.poisson(lambda_j * dt, num_sims)
        J = np.random.normal(mu_j, sigma_j, num_sims)
        jump = J * dN
        
        #Combine
        paths[:, i] = paths[:, i-1] * np.exp(diffusion + jump)
        
    return paths

In [131]:
def Option_pricing(Spot, Strike, T, r, q, sigma, lambda_j, mu_j, sigma_j, num_sims, option_type='call'):
    # Simulate stock price paths
    N = int(T*252)
    paths = jump_diffusion_simulation(Spot, r, q, sigma, lambda_j, mu_j, sigma_j, T, N, num_sims)
    
    # Calculate terminal stock prices
    S_T = paths[:, -1]
    
    # Calculate payoffs
    if option_type == 'call':
        payoffs = np.maximum(S_T - Strike, 0)
        Probability = np.mean(S_T > Strike)
    elif option_type == 'put':
        payoffs = np.maximum(Strike - S_T, 0)
        Probability = np.mean(S_T < Strike)
    else:
        print('Choose the either call or put, thank you.')
        return

    
    # Calculate option price
    option_price = np.exp(-r * T) * np.mean(payoffs)
    
    # Calculate standard error
    option_se = np.exp(-r * T) * np.std(payoffs) / np.sqrt(num_sims)
    
    return option_price, option_se, Probability

# Example usage:
Spot = 100
Strike = 120
T = 1
r = 0.05
q = 0.02
sigma = 0.1
lambda_j = 0.1
mu_j = 0.05
sigma_j = 0.1
num_sims = 2**13
price, se, prob = Option_pricing(Spot, Strike, T, r, q, sigma, lambda_j, mu_j, sigma_j, num_sims, option_type='put')
print(f"Option Price: {price:.4f}")
print(f"Standard Error: {se:.4f}")
print(f"Probability of S > K: {prob:.4f}")

Option Price: 16.1001
Standard Error: 0.1051
Probability of S > K: 0.9254
